In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DreamTalk 사용을 위한 환경 설정

In [ ]:
# DreamTalk
%cd /content
!rm -rf dreamtalk
!git clone -b dev https://github.com/camenduru/dreamtalk
%cd /content/dreamtalk
!pip install -q yacs av

# 파일 따로 첨부
!cp /content/drive/MyDrive/dreamtalk/denoising_network.pth /content/dreamtalk/checkpoints/denoising_network.pth
!cp /content/drive/MyDrive/dreamtalk/renderer.pt /content/dreamtalk/checkpoints/renderer.pt


/content
Cloning into 'dreamtalk'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 164 (delta 7), reused 3 (delta 3), pack-reused 144
Receiving objects: 100% (164/164), 28.14 MiB | 13.46 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/dreamtalk


# Run inference_for_demo_video.py
폴더 안의 audio, style, image 활용 혹은 직접 파일을 넣어 지정 가능


output_video 라고 동영상 이름 지정

In [ ]:
!python inference_for_demo_video.py \
--wav_path data/audio/acknowledgement_english.m4a \
--style_clip_path data/style_clip/3DMM/M030_front_neutral_level1_001.mat \
--pose_path data/pose/RichardShelby_front_neutral_level1_001.mat \
--image_path data/src_img/uncropped/male_face.png \
--cfg_scale 1.0 \
--max_gen_len 30 \
--output_name output_video

print("생성 완료")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# GFPGAN 이용을 위한 환경 재설정

In [ ]:
%cd ..

# GFPGAN
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd /content/GFPGAN

!pip install basicsr --quiet
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan --quiet

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

# colab 환경에서 진행하기 위해 torchvision downscale (필수)
!pip uninstall torch torchvision -y
!pip install torch==1.13.0 torchvision==0.14.0 -f https://download.pytorch.org/whl/cu121/torch_stable.html


/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 527 (delta 170), reused 155 (delta 154), pack-reused 314
Receiving objects: 100% (527/527), 5.38 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (281/281), done.
/content/GFPGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-an

# Upscale by GFPGAN

생성한 video를 복사한 후, GFPGAN 활용을 위해 frame 단위로 쪼개어 영상을 저장

In [ ]:
import os
import shutil
import cv2

# 비디오 파일의 경로를 지정
source_video_path = "/content/dreamtalk/output_video/output_video.mp4"
upload_folder = "video/inputs"

# 기존 폴더가 있으면 삭제하고 새로 생성
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)

# 비디오 파일을 지정된 폴더로 복사
filename = os.path.basename(source_video_path)
video_path = os.path.join(upload_folder, filename)
shutil.copy(source_video_path, video_path)
print(f'Copied {source_video_path} to {video_path}')

# 비디오에서 프레임을 추출
video_capture = cv2.VideoCapture(video_path)
frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))

frame_folder = os.path.join(upload_folder, os.path.splitext(filename)[0])
os.makedirs(frame_folder, exist_ok=True)

frame_count = 0
while True:
    success, frame = video_capture.read()
    if not success:
        break

    # 프레임 저장
    frame_filename = f"{frame_count:04d}.jpg"
    frame_path = os.path.join(frame_folder, frame_filename)
    cv2.imwrite(frame_path, frame)
    frame_count += 1

video_capture.release()
print(f'Frames saved to {frame_folder}')


Copied /content/dreamtalk/output_video/output_video.mp4 to video/inputs/output_video.mp4
Frames saved to video/inputs/output_video


In [ ]:
input_folder = frame_folder
output_folder = os.path.join('video/output/images/', os.path.splitext(filename)[0])

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)

# Run inference_gfpgan.py
프레임 단위로 화질 향상

In [ ]:
!python inference_gfpgan.py -i {input_folder} -o {output_folder} -v 1.3 -s 2

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing 0000.jpg ...
	Tile 1/1
Processing 0001.jpg ...
	Tile 1/1
Processing 0002.jpg ...
	Tile 1/1
Processing 0003.jpg ...
	Tile 1/1
Processing 0004.jpg ...
	Tile 1/1
Processing 0005.jpg ...
	Tile 1/1
Processing 0006.jpg ...
	Tile 1/1
Processing 0007.jpg ...
	Tile 1/1
Processing 0008.jpg ...
	Tile 1/1
Processing 0009.jpg ...
	Tile 1/1
Processing 0010.jpg ...
	Tile 1/1
Processing 0011.jpg ...
	Tile 1/1
Processing 0012.jpg ...
	Tile 1/1
Processing 0013.jpg ...
	Tile 1/1
Processing 0014

In [ ]:
file_name = os.path.splitext(filename)[0]
output_folder = f'video/output/images/{file_name}/restored_imgs'
output_video_path = f'video/output/{filename}'
#output_folder

In [ ]:
import cv2
import os
from google.colab import files

output_folder = f'video/output/images/{file_name}/restored_imgs'
output_video_path = f'video/output/{filename}'

frames = [os.path.join(output_folder, frame) for frame in os.listdir(output_folder) if frame.endswith('.jpg')]
frames.sort()
frame = cv2.imread(frames[0])
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 25.0, (width, height))

for frame_path in frames:
    img = cv2.imread(frame_path)
    out.write(img)

out.release()

# 프레임 단위로 쪼개면서 음성 유실 -> 다시 합성 과정

audio_path = "/content/dreamtalk/data/audio/acknowledgement_english.m4a"
final_path = f'/content/GFPGAN/video/output/audio_{filename}.mp4'

def add_audio_to_video(video_path, audio_path, final_path):
    cmd = f'ffmpeg -i {video_path} -i {audio_path} -c:v copy -c:a aac -strict experimental {final_path}'
    os.system(cmd)

add_audio_to_video(output_video_path, audio_path, final_path)

# output 다운로드
files.download(final_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>